In [2]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [1]:
import os
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR) 
import numpy as np
import scipy.io as sio
import random
import cv2
from termcolor import colored
import utils 
np.seterr(divide='ignore', invalid='ignore')

import image_components
import pose_components
import matplotlib.pyplot as plt
import dataloader
from test_data_loader import get_h36_test, data_loader_h36
from hyperparams import Hyperparameters
import multiprocessing as mp 
from sklearn.utils import shuffle

subject  S1
subject  S5
subject  S6
subject  S7
subject  S8
subject  S9
subject  S11


In [9]:
H = Hyperparameters ()
os.environ['CUDA_VISIBLE_DEVICES'] = H.cuda_device_id
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)

In [10]:
h36_test_data = get_h36_test()
h36_test_dataset = tf.data.Dataset.from_tensor_slices((h36_test_data))
dataset_human_test = h36_test_dataset.map(lambda z: tf.py_func(data_loader_h36, [z], [tf.float32, \
        tf.float32]), 32)
dataset_human_test = dataset_human_test.batch(H.batch_size)
dataset_human_test = dataset_human_test.prefetch(2)
test_iterator = dataset_human_test.make_initializable_iterator()
test_img, test_pose = test_iterator.get_next()

Length of H3.6M test set: 2181


In [11]:
image_ph = tf.placeholder(tf.float32 , shape = [None, H.image_size,H.image_size ,3] , name = 'image_ph')
pose_ph = tf.placeholder(tf.float32 , shape = [None,17,3] , name = 'pose_gt_ph')

def get_image_to_latent(input_image): # input_image shape:[None,224,224,3]
    resnet_out = image_components.create_network(input_image,'')
    resnet_params = image_components.get_network_params('')
    fc_embedding = image_components.embedding_branch(resnet_out)
    fc_embedding_params = image_components.get_network_params('embedding')
    outputs = {
        'resnet_params' :resnet_params,
        'im_embed' :fc_embedding,
        'fc_embedding_params':fc_embedding_params
    }
    return outputs
    
cnn_outputs = get_image_to_latent(image_ph)
embed=cnn_outputs['im_embed']
pose_pred = pose_components.apply_pose_decoder(embed)

loss = tf.reduce_mean(tf.abs(pose_pred - pose_ph))
loss_summary = tf.summary.scalar("pose_pred_loss",loss)
merge_summary = tf.summary.merge_all()

resnet_params = cnn_outputs['resnet_params']
fc_embedding_params = cnn_outputs['fc_embedding_params']
pose_decoded_params = pose_components.get_network_params('Decoder_net')

opt = tf.train.AdamOptimizer(H.learning_rate).minimize(loss = loss,
                                                       var_list = resnet_params + fc_embedding_params)


sess.run(tf.global_variables_initializer())


resnet_branch_weights = tf.train.Saver(resnet_params,max_to_keep=5)
embedding_branch_weights = tf.train.Saver(fc_embedding_params,max_to_keep=5)
decoder_branch_weights = tf.train.Saver(pose_decoded_params,max_to_keep=5)

In [12]:
tf.train.Saver(resnet_params).restore(sess,tf.train.latest_checkpoint('./weights/resnet/'))
print (colored("restored resnet branch weights","yellow"))

tf.train.Saver(fc_embedding_params).restore(sess,tf.train.latest_checkpoint('./weights/embedding_branch/'))
print (colored("Restored Embedding Branch weights","green"))

tf.train.Saver(pose_decoded_params).restore(sess,'../pretrained_weights/pose_encoder_decoder/decoder_iter-1475001')
print (colored("Restored Decoder weights","blue"))

restored resnet branch weights
Restored Embedding Branch weights
Restored Decoder weights


In [14]:

loss_list = []
sess.run(test_iterator.initializer)
while True:
    try:
        images, pose_3d =  sess.run([test_img, test_pose])
        val_loss,pred_3d = sess.run([loss,pose_pred], feed_dict = {image_ph : images , pose_ph : pose_3d})
        loss_list.append(val_loss) 
    except tf.errors.OutOfRangeError:
        print("Val loss : ",sum(loss_list)/len(loss_list))
        break
print (sum(loss_list)/len(loss_list))

Val loss :  241.15142313639322
241.15142313639322
